In [1]:
import numpy as np
import torch
import pickle
from denoising_diffusion_pytorch import Unet1D, GaussianDiffusion1D, Trainer1D, Dataset1D
from torch.utils.data import Dataset, DataLoader
from multiprocessing import cpu_count

In [2]:
def cycle(dl):
    while True:
        for data in dl:
            yield data

# test real data

In [3]:
dataset_name = '2_29_sine_dataset'
# training_seq = torch.rand(64, 32, 128) # features are normalized from 0 to 1
with open(f'./dataset/{dataset_name}/data.pkl', 'rb') as f:
    dataset = pickle.load(f)

In [4]:
dl = DataLoader(dataset, batch_size = 32, shuffle = True, pin_memory = True, num_workers = cpu_count())

/om2/user/qiyao/.conda/envs/env1/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 48 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [6]:
dl = cycle(dl)
data = next(dl)

In [7]:
img, labels = data

In [8]:
img.shape

torch.Size([32, 128])

In [9]:
img.unsqueeze(1).shape

torch.Size([32, 1, 128])

In [10]:
img.shape

torch.Size([32, 128])

In [21]:
img_double = img.type(torch.float)

In [22]:
img_double.dtype

torch.float32

# test synthetic data

In [4]:
training_seq = torch.rand(4, 32, 128) # features are normalized from 0 to 1
dataset = Dataset1D(training_seq) 
dl = DataLoader(dataset, batch_size = 32, shuffle = True, pin_memory = True, num_workers = cpu_count())
dl = cycle(dl)

/om2/user/qiyao/.conda/envs/env1/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 48 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [5]:
data = next(dl)

In [7]:
data.shape

torch.Size([4, 32, 128])

In [14]:
model = Unet1D(
    dim = 64,
    dim_mults = (1, 2, 4, 8),
    channels = 32
)

diffusion = GaussianDiffusion1D(
    model,
    seq_length = 128,
    timesteps = 1000,
    objective = 'pred_noise'
)
dataset_name = '2_29_sine_dataset'
# training_seq = torch.rand(64, 32, 128) # features are normalized from 0 to 1
with open(f'./dataset/{dataset_name}/data.pkl', 'rb') as f:
    dataset = pickle.load(f)
# dataset = Dataset1D(training_seq)  # this is just an example, but you can formulate your own Dataset and pass it into the `Trainer1D` below

# loss = diffusion(dataset)
# loss.backward()

# Or using trainer

trainer = Trainer1D(
    diffusion,
    dataset = dataset,
    train_batch_size = 32,
    train_lr = 8e-5,
    train_num_steps = 700000,         # total training steps
    gradient_accumulate_every = 2,    # gradient accumulation steps
    ema_decay = 0.995,                # exponential moving average decay
    amp = True,                       # turn on mixed precision
)
trainer.train()

# after a lot of training

# sampled_seq = diffusion.sample(batch_size = 4)
# sampled_seq.shape # (4, 32, 128)

: 